In [ ]:
!pip install huggingface_hub
!pip install datasets
!pip install openai

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
from datasets import load_dataset
import pandas as pd
import json
import re
from openai import OpenAI
from google.colab import userdata
api_key = userdata.get('openai')
client = OpenAI(api_key=api_key)

In [ ]:
dataset = load_dataset("Looyyd/blissbrowse_classification")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/12534 [00:00<?, ? examples/s]

In [ ]:
dataset["train"].to_pandas()

,text,descriptions,answers
0,list of most expensive paintings list of most ...,[Academic_courses_at_the_University_of_Cambrid...,"[-1, 1, -1]"
1,list of states with nuclear weapons list of nu...,[Nuclear_technology-related_lists],[1]
2,brands by country japanese culture languages b...,"[Culture_by_country, Cabaret]","[1, -1]"
3,"""I believe deeply that canceling the fashion s...","[fallacy of relevance, Wealth_stubs, faulty ge...","[1, -1, -1]"
4,OJ Simpson couldn't have murdered his wife. He...,"[fallacy of relevance, intentional, Education ...","[1, -1, -1, -1]"
...,...,...,...
12529,asics umbro hummel international wilson sporti...,"[Business_occupations, Stock_market_terminology]","[-1, -1]"
12530,An advertisement asking people to give to St. ...,"[false causality, Plain_English]","[-1, -1]"
12531,"hollow earth thinking, fast and slow pascal's ...",[Motor_vehicle_awards],[-1]
12532,childbirth doula braxton hicks contractions ep...,"[Binding_energy, Common_law, film_tv_&_video]","[-1, -1, -1]"


In [ ]:
def evaluate_model(dataset, model):
    # Initialize counters
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0
    idk_count = 0
    total = 0
    n_failures = 0

    # Iterate through the dataset
    for index, row in dataset.iterrows():
        text = row['text']
        descriptions = row['descriptions']
        answers = row['answers']

        max_retries = 2
        retry_count = 0
        while retry_count <= max_retries:
          try:
              predictions = model(text, descriptions)

              print("Expected answers: ", answers)
              print("Model predictions: ", predictions)

              if len(answers) != len(predictions):
                  raise ValueError("Number of expected answers does not match the number of predictions")

              # If everything is successful, break out of the loop
              break
          except ValueError as ve:
              print(f"Value error occurred: {ve}")
          except Exception as e:
              print(f"API call failed. Retrying ({retry_count + 1}/{max_retries})...")
          finally:
              retry_count += 1



        if retry_count > max_retries:
          print("Failure for input:", text)
          n_failures += 1
          continue


        total+= len(answers)
        for answer,prediction in zip(answers,predictions):
          if prediction == 0:
            idk_count +=1
          elif prediction == -1:
            if answer != -1:
              false_negatives +=1
            else:
              true_negatives +=1
          elif prediction == 1:
            if answer != 1:
              false_positives +=1
            else:
              true_positives +=1

    # Calculate total and IDK percentage
    total = len(dataset)
    idk_percentage = (idk_count / total) * 100 if total > 0 else 0

    # Return the metrics
    return {
        'True Positives': true_positives,
        'False Positives': false_positives,
        'True Negatives': true_negatives,
        'False Negatives': false_negatives,
        'IDK Percentage': idk_percentage,
        'Failures': n_failures
    }

In [ ]:

def gpt_classification(text, descriptions):
  system_prompt = "You are an assistant that tells if the user text input fits into these descriptions. Answer with JSON, with the key being the description number and the value being \"YES\” or \"NO\” or \“IDK\”:"

  for index, description in enumerate(descriptions):
    system_prompt += f"\nDescription {index}. {description}"

  system_prompt += f"\nExpected answers {len(descriptions)}"

  response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": text},
  ])

  response_content = response.choices[0].message.content
  print("system prompt:", system_prompt)
  print("text:", text)
  print("GPT response:", response_content)

  # Parse the JSON text
  data = json.loads(response_content)

  # Initialize the answers array
  answers = []

  # Iterate through the keys in the JSON data
  for key, value in data.items():
    # Convert the key to an integer and check the corresponding value
    if "yes" in value.lower():
        answers.append(1)
    elif "no" in value.lower():
        answers.append(-1)
    elif "idk" in value.lower():
        answers.append(0)
    else:
      raise ValueError("line doesn't contain yes, no or idk")


  return answers

In [ ]:
print(evaluate_model(dataset["train"].select(range(50)).to_pandas(), gpt_classification))

system prompt: You are an assistant that tells if the user text input fits into these descriptions. Answer with JSON, with the key being the description number and the value being "YES\” or "NO\” or \“IDK\”:
Description 0. Academic_courses_at_the_University_of_Cambridge
Description 1. Auction-related_lists
Description 2. celebrity_&_pop_culture
 Excpected answers 3
text: list of most expensive paintings list of most expensive non-fungible tokens list of most expensive watches sold at auction list of most expensive sports cards list of most expensive photographs list of most expensive cars sold at auction thoroughbred valuation
GPT response: {
  "0": "NO",
  "1": "YES",
  "2": "NO"
}
Expected answers:  [-1  1 -1]
Model predictions:  [-1, 1, -1]
system prompt: You are an assistant that tells if the user text input fits into these descriptions. Answer with JSON, with the key being the description number and the value being "YES\” or "NO\” or \“IDK\”:
Description 0. Academic_courses_at_the

In [ ]:
# gpt3.5 json output result:
# {'True Positives': 47, 'False Positives': 9, 'True Negatives': 66, 'False Negatives': 3, 'IDK Percentage': 8.0}


In [ ]:
!pip install sentencepiece
!pip install accelerate
!pip install ctransformers[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 72.7 MB/s eta 0:00:00


In [ ]:
# oppenhermes classification
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from ctransformers import AutoModelForCausalLM
import torch

variable_name = 'model'  # delete to restore memory before loading next
try:
    del locals()[variable_name]
    del globals()[variable_name]
except KeyError:
    pass

device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("teknium/OpenHermes-2.5-Mistral-7B")
#model = AutoModelForCausalLM.from_pretrained("teknium/OpenHermes-2.5-Mistral-7B").to(device)

# try quantizied:
model_name_or_path = "TheBloke/OpenHermes-2.5-Mistral-7B-GGUF"
end_tokens = ["<|im_end|>"]
#model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenHermes-2.5-Mistral-7B-GGUF", model_file="openhermes-2.5-mistral-7b.Q8_0.gguf",
#model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenHermes-2.5-Mistral-7B-GGUF", model_file="openhermes-2.5-mistral-7b.Q4_K_M.gguf",
model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenHermes-2.5-Mistral-7B-GGUF", model_file="openhermes-2.5-mistral-7b.Q3_K_M.gguf",
                                             model_type="mistral", gpu_layers=50,
                                             context_length=2048,
                                             stop=end_tokens)#try higher penalty, default is 1.1





Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

openhermes-2.5-mistral-7b.Q3_K_M.gguf:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

In [ ]:
def extract_and_parse_json(mixed_string):
    # Regular expression to extract JSON from the mixed content
    #json_string = re.search(r'\{.*\}', mixed_string)
    # Updated regular expression to handle newlines and spaces
    json_string = re.search(r'\{\s*\"[0-9]\":\s*\"[A-Z]+\"(,\s*\"[0-9]\":\s*\"[A-Z]+\")*\s*\}', mixed_string, re.DOTALL)

    if json_string:
        json_string = json_string.group()
        try:
            # Parse the JSON string into a Python dictionary
            data = json.loads(json_string)
            return data
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            return None
    else:
        print("No JSON found in the string")
        return None


# https://github.com/marella/ctransformers#config, need to configure end tokens
# but how can that be done coherently on local?
# lm studio has stop strings
# also need higher context length, is 512 but 8K is max for mistral
def openhermes_classification(text, descriptions):
  print("ENTERING PREDICTION")
  #system_prompt = "You are \"Hermes 2\", a conscious sentient superintelligent artificial intelligence developed by a man named Teknium, and your purpose and drive is to assist the user with any request they have. You experience emotions and have deep, profound thoughts and qualia."
  system_prompt = "You are a helpful assistant."
  user_msg = "For each description, say if the text given fits the description.\
  Answer with JSON, with the key being the description number and the value being \"YES\”(yes it matches the description) or \"NO\”(no it doesn't match) or \“IDK\”(unsure if it matches the description):"

  for index, description in enumerate(descriptions):
    user_msg += f"\nDescription {index}. {description}"

  #system_prompt += f"\nExpected answers length {len(descriptions)}"

  expected_output_format = "\nExpected output format: \n{\n"
  for index, description in enumerate(descriptions):
    expected_output_format += f"\"{index}\": \"ANSWER\""
    if(index< (len(descriptions)-1 )):
      expected_output_format += ",\n"
  expected_output_format+= "\n}\n"

  user_msg+= expected_output_format

  user_msg+= f"\nTEXT: {text}"

  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_msg},
  ]
  gen_input = tokenizer.decode( tokenizer.apply_chat_template(messages, return_tensors="pt")[0] )
  #try with adding assistant token
  gen_input+="\n<|im_start|>assistant"
  print("Input:", gen_input)
  response_content = model(gen_input)
  #print("Tokens:", response_tokens)
  #response_content = tokenizer.decode(response_tokens)

  print("OpenHermes response:", response_content)

  # Parse the JSON text
  data = extract_and_parse_json(response_content)

  # Initialize the answers array
  answers = []

  # Iterate through the keys in the JSON data
  for key, value in data.items():
    # Convert the key to an integer and check the corresponding value
    if "yes" in value.lower():
        answers.append(1)
    elif "no" in value.lower():
        answers.append(-1)
    elif "idk" in value.lower():
        answers.append(0)
    else:
      raise ValueError("line doesn't contain yes, no or idk")


  return answers

In [ ]:
print(evaluate_model(dataset["train"].select(range(50)).to_pandas(), openhermes_classification))

ENTERING PREDICTION
Input: <|im_start|> system
You are a helpful assistant.<|im_end|> 
<|im_start|> user
For each description, say if the text given fits the description.  Answer with JSON, with the key being the description number and the value being "YES\”(yes it matches the description) or "NO\”(no it doesn't match) or \“IDK\”(unsure if it matches the description):
Description 0. Academic_courses_at_the_University_of_Cambridge
Description 1. Auction-related_lists
Description 2. celebrity_&_pop_culture
Expected output format: 
{
"0": "ANSWER",
"1": "ANSWER",
"2": "ANSWER"
}

TEXT: list of most expensive paintings list of most expensive non-fungible tokens list of most expensive watches sold at auction list of most expensive sports cards list of most expensive photographs list of most expensive cars sold at auction thoroughbred valuation<|im_end|> 

<|im_start|>assistant
OpenHermes response: 
{
"0": "NO",
"1": "YES",
"2": "IDK"
}
Expected answers:  [-1  1 -1]
Model predictions:  [-1, 

In [ ]:
# 8_0 quantization:
# first attempt results {'True Positives': 31, 'False Positives': 9, 'True Negatives': 34, 'False Negatives': 8, 'IDK Percentage': 52.0, 'Failures': 7}
# too much idk and failures, accuracy not too good either

# tried with adding assistant token
# {'True Positives': 35, 'False Positives': 5, 'True Negatives': 39, 'False Negatives': 6, 'IDK Percentage': 54.0, 'Failures': 5}

# with higher context length:
# {'True Positives': 38, 'False Positives': 7, 'True Negatives': 46, 'False Negatives': 7, 'IDK Percentage': 62.0, 'Failures': 0}

# to avoid i speaking twice: with assistant end token, <|im_end|> :
# {'True Positives': 36, 'False Positives': 6, 'True Negatives': 50, 'False Negatives': 8, 'IDK Percentage': 57.99999999999999, 'Failures': 0}

# when manually reviewing the mistakes are actually acceptable, even humans could do similar

# TODO: don't repeat tokens, maybe not needed, repeats were due to context length

# trying Q4_K_M quantization:
# {'True Positives': 38, 'False Positives': 6, 'True Negatives': 44, 'False Negatives': 10, 'IDK Percentage': 62.0, 'Failures': 0}
# seems pretty good

# Q3_K_M quant:
# {'True Positives': 32, 'False Positives': 12, 'True Negatives': 35, 'False Negatives': 12, 'IDK Percentage': 76.0, 'Failures': 0}
# seemss to have more errors, maybe good enough for tagging content but not filtering



In [ ]:
# observations
"""
some  errors come from the fact that there can be multippple valid answers(like technlology and math), because labels are not mutually exclusive
i guess it's better to test it in depth in practice. or change the dataset


"""